## Import important stuff

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn import linear_model, svm, tree
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from Backend.Evaluation.metrics import compute_evaluation_metrics
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import xgboost as xgb


### def create tuple

#### delete districts, transform variant, delet others

In [2]:
 # 1) separate beta, delete districts, transform variant
df = pd.read_csv("all_matrix_data_v2.csv")
df

,Unnamed: 0,week,policy_index,variant,mobility,temperature,wind,infections,beta,beta_t_minus_1,district
0,0,2,62.500000,Other,-41.714286,4.885714,24.057143,4.897959,2.621485e-01,0.405904,Landkreis St. Wendel
1,1,3,63.778571,Other,-36.000000,5.971429,14.714286,3.326531,1.550497e-01,0.269774,Landkreis St. Wendel
2,2,4,65.180000,Other,-31.142857,16.085714,11.900000,1.122449,3.379712e-18,0.278678,Landkreis St. Wendel
3,3,5,65.180000,Other,-31.857143,12.357143,12.542857,0.500000,1.574978e-02,0.002448,Landkreis St. Wendel
4,4,6,65.180000,Other,-23.571429,13.685714,19.585714,0.704082,5.966302e-01,0.014601,Landkreis St. Wendel
...,...,...,...,...,...,...,...,...,...,...,...
37195,88,90,81.761429,B.1.617.2,1.857143,2.185714,14.614286,165.051020,5.811076e-01,0.612582,Wartburgkreis
37196,89,91,83.930000,B.1.617.2,6.000000,0.857143,9.814286,145.214286,4.988738e-01,0.586679,Wartburgkreis
37197,90,92,83.930000,B.1.617.2,6.571429,5.028571,7.842857,105.612245,3.810974e-01,0.542123,Wartburgkreis
37198,91,93,83.930000,B.1.617.2,-2.714286,-1.842857,10.771429,83.377551,4.671758e-01,0.425633,Wartburgkreis


In [3]:
#transform variant
df_1 = pd.get_dummies(df['variant'])
df_1

,0,B.1.1.7,B.1.617.2,Other
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
37195,0,0,1,0
37196,0,0,1,0
37197,0,0,1,0
37198,0,0,1,0


In [4]:
df_2 = df.drop(columns='variant')
df = pd.concat([df_1, df_2], axis=1)
#delete districts
df = df.drop(columns='district')
df

,0,B.1.1.7,B.1.617.2,Other,Unnamed: 0,week,policy_index,mobility,temperature,wind,infections,beta,beta_t_minus_1
0,0,0,0,1,0,2,62.500000,-41.714286,4.885714,24.057143,4.897959,2.621485e-01,0.405904
1,0,0,0,1,1,3,63.778571,-36.000000,5.971429,14.714286,3.326531,1.550497e-01,0.269774
2,0,0,0,1,2,4,65.180000,-31.142857,16.085714,11.900000,1.122449,3.379712e-18,0.278678
3,0,0,0,1,3,5,65.180000,-31.857143,12.357143,12.542857,0.500000,1.574978e-02,0.002448
4,0,0,0,1,4,6,65.180000,-23.571429,13.685714,19.585714,0.704082,5.966302e-01,0.014601
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37195,0,0,1,0,88,90,81.761429,1.857143,2.185714,14.614286,165.051020,5.811076e-01,0.612582
37196,0,0,1,0,89,91,83.930000,6.000000,0.857143,9.814286,145.214286,4.988738e-01,0.586679
37197,0,0,1,0,90,92,83.930000,6.571429,5.028571,7.842857,105.612245,3.810974e-01,0.542123
37198,0,0,1,0,91,93,83.930000,-2.714286,-1.842857,10.771429,83.377551,4.671758e-01,0.425633


In [5]:
# Drop "other" variant column to prevent dummy variable trap:
df = df.drop(columns='Other', axis=1)
# Drop zero column:
df = df.drop(columns='0', axis=1)

#### delete infections = 0

In [6]:
df.drop(df[df['infections'] == 0.0].index, inplace=True)
df.reset_index(inplace=True, drop=True)

In [7]:
df_v4 = df.copy()

In [8]:
df_v4['temperature'] = df.groupby("week").transform(lambda x: x.fillna(x.mean()))['temperature']
df_v4['wind'] = df.groupby("week").transform(lambda x: x.fillna(x.mean()))['wind']

In [9]:
df = df_v4
df

,B.1.1.7,B.1.617.2,Unnamed: 0,week,policy_index,mobility,temperature,wind,infections,beta,beta_t_minus_1
0,0,0,0,2,62.500000,-41.714286,4.885714,24.057143,4.897959,2.621485e-01,0.405904
1,0,0,1,3,63.778571,-36.000000,5.971429,14.714286,3.326531,1.550497e-01,0.269774
2,0,0,2,4,65.180000,-31.142857,16.085714,11.900000,1.122449,3.379712e-18,0.278678
3,0,0,3,5,65.180000,-31.857143,12.357143,12.542857,0.500000,1.574978e-02,0.002448
4,0,0,4,6,65.180000,-23.571429,13.685714,19.585714,0.704082,5.966302e-01,0.014601
...,...,...,...,...,...,...,...,...,...,...,...
36771,0,1,88,90,81.761429,1.857143,2.185714,14.614286,165.051020,5.811076e-01,0.612582
36772,0,1,89,91,83.930000,6.000000,0.857143,9.814286,145.214286,4.988738e-01,0.586679
36773,0,1,90,92,83.930000,6.571429,5.028571,7.842857,105.612245,3.810974e-01,0.542123
36774,0,1,91,93,83.930000,-2.714286,-1.842857,10.771429,83.377551,4.671758e-01,0.425633


#### delete small betas

In [10]:
df.drop(df[df['beta'] <= 0.025].index, inplace=True)
df.reset_index(inplace=True, drop=True)

#### create beta, delete bate from dataframe

In [11]:
#dataframe for labels
beta = df['beta']

#delete beta
df = df.drop(columns='beta')

#### delete columns week and unnamed

In [12]:
df = df.drop(columns='week')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [13]:
df

,B.1.1.7,B.1.617.2,policy_index,mobility,temperature,wind,infections,beta_t_minus_1
0,0,0,62.500000,-41.714286,4.885714,24.057143,4.897959,0.405904
1,0,0,63.778571,-36.000000,5.971429,14.714286,3.326531,0.269774
2,0,0,65.180000,-23.571429,13.685714,19.585714,0.704082,0.014601
3,0,0,69.342857,-27.428571,10.585714,15.171429,0.622449,0.416047
4,0,0,61.268571,-12.428571,9.585714,15.528571,0.122449,0.005292
...,...,...,...,...,...,...,...,...
34866,0,1,81.761429,1.857143,2.185714,14.614286,165.051020,0.612582
34867,0,1,83.930000,6.000000,0.857143,9.814286,145.214286,0.586679
34868,0,1,83.930000,6.571429,5.028571,7.842857,105.612245,0.542123
34869,0,1,83.930000,-2.714286,-1.842857,10.771429,83.377551,0.425633


#### Standadizing

In [14]:
#save the given model
def save_standardizing_model(model):
    filename = 'standardizing_model.sav'
    pickle.dump(model, open(filename, 'wb'))

In [15]:
# Standardize:
scaler_obj = StandardScaler()

# Split explanatory columns from beta:
explanatory_cols = df.drop(columns=['beta_t_minus_1'], axis=1)
beta_cols = df[['beta_t_minus_1']]

# Apply scaling:
scaled_explanatory_cols = pd.DataFrame(scaler_obj.fit_transform(explanatory_cols), index=explanatory_cols.index, columns=explanatory_cols.columns)

# Merge both together again:
df = pd.concat([scaled_explanatory_cols, beta_cols], axis=1)
df
save_standardizing_model(scaler_obj)

#### split into test and training data

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df, beta, shuffle=True, train_size=0.75, random_state=1)

In [17]:
df.to_csv("df.csv")
beta.to_csv("beta.csv")
X_test.to_csv("x_test.csv")
X_train.to_csv("x_train.csv")
y_test.to_csv("y_test.csv")
y_train.to_csv("y_train.csv")
X_test, X_train, y_test, y_train

(        B.1.1.7  B.1.617.2  policy_index  mobility  temperature      wind  \
 27985  1.702719  -0.643395      1.021763 -0.249863     1.430504 -1.188040   
 11433 -0.587296   1.554256     -1.003078  0.263379     0.030636 -0.060999   
 7535  -0.587296   1.554256      0.358885  2.637122     0.618367 -0.800241   
 12155 -0.587296  -0.643395     -0.593654  0.602485     0.165280  0.950792   
 27209  1.702719  -0.643395      0.790591  0.804116    -1.497464  0.221771   
 ...         ...        ...           ...       ...          ...       ...   
 30772 -0.587296  -0.643395     -1.589363  0.226719     1.396308 -0.145830   
 8547  -0.587296  -0.643395     -0.593654  0.034253     0.308472  2.088053   
 1491   1.702719  -0.643395      0.406233 -1.642948    -0.937517  2.859612   
 21117  1.702719  -0.643395      0.375596 -0.717280    -1.076435  3.400915   
 19279 -0.587296  -0.643395      0.747222 -1.083881    -1.016593 -0.650777   
 
        infections  beta_t_minus_1  
 27985   -0.349154       

### run all models to find best one

#### train and run the models

In [18]:
def run_train_test(X_train, y_train, X_test, y_test, modeltype):
    model = ml_training(X_train, y_train, modeltype)
    return(ml_testing(X_test, y_test, model)), model


#### ML training

In [19]:
def ml_training(x, y, model):
    if model == "linear_regression":
        reg = linear_model.LinearRegression()
    elif model == "lasso":
        reg = linear_model.Lasso(alpha=0.1)
    elif model == "support_vector_machine":
        reg = svm.SVR()
    elif model == "linear_regression_tree":
        reg = tree.DecisionTreeRegressor()
    elif model == "ensemble_method_adaboost":
        y_array = y.to_numpy()
        y_array = y_array.ravel()
        reg = AdaBoostRegressor(random_state=0, n_estimators=100)
        reg.fit(x, y_array)
        return reg
    elif model == "neural_network":
        y_array = y.to_numpy()
        y_array = y_array.ravel()
        reg = MLPRegressor(random_state=1, max_iter=500).fit(x, y_array)
        return reg
    elif model == "random_forest_regressor":
        reg = RandomForestRegressor(n_estimators=10, random_state=0, max_depth=2)
        y_array = y.to_numpy()
        y_array = y_array.ravel()
        reg = reg.fit(x, y_array)
        return reg
    elif model == "XG_boost":
        xgb_model = xgb.XGBRegressor(verbosity=0)
        xgb_model.name = 'XGBoostRegressor'
        reg = xgb_model.fit(X_train, y_train)
        return reg
    reg.fit(x, y)
    return reg

#### ML testing

In [20]:
#test the fitted model
def ml_testing(X_test, y_test, model):
    pred = model.predict(X_test)
    scores = compute_evaluation_metrics(y_test,pred)
    return(scores["rmse"])

In [21]:
#save the given model
def save_model(model):
    filename = 'finalized_model.sav'
    pickle.dump(model, open(filename, 'wb'))

In [22]:
#load a previous saved model
def load_model():
    # load the model from disk
    filename = 'finalized_model.sav'
    loaded_model = pickle.load(open(filename, 'rb'))
    loaded_model
    return loaded_model

In [23]:
#test the fitted model: coompare pred and beta t-1
def ml_testing_beta(X_test, y_test):
    beta_t_1 = X_test['beta_t_minus_1']
    scores_beta = compute_evaluation_metrics(y_val=y_test, y_pred=beta_t_1)
    return scores_beta["rmse"]

In [29]:
#run all model types to find best model
rmse = []
min = 100000
metrics =["linear_regression", "lasso", "support_vector_machine", "linear_regression_tree", "ensemble_method_adaboost", "neural_network", "random_forest_regressor", "XG_boost"]
for i in range(len(metrics)):
    rmse.append(run_train_test(X_train, y_train, X_test, y_test, metrics[i]))
for i in range(len(rmse)):
    if rmse[i][0] < min:
        min = rmse[i][0]
        best_model = rmse[i][1]


### Store rmse results


In [50]:
save_model(best_model)
df = pd.DataFrame(rmse)
beta_rmse = []
beta_rmse.append((ml_testing_beta(X_test, y_test), 'beta_t_minus_1'))
df = df.append(beta_rmse)
df.columns = ['RMSE', 'Model']
df.to_csv('rmse.csv')

(       RMSE                                              Model
 0  3.055869                                 LinearRegression()
 1  3.070969                                   Lasso(alpha=0.1)
 2  4.129604                                              SVR()
 3  7.778841                            DecisionTreeRegressor()
 4  2.983452  (DecisionTreeRegressor(max_depth=3, random_sta...
 5  2.792079         MLPRegressor(max_iter=500, random_state=1)
 6  3.026785  (DecisionTreeRegressor(max_depth=2, max_featur...
 7  3.974076  XGBRegressor(base_score=0.5, booster='gbtree',...
 0  2.979921                                     beta_t_minus_1,
 [(2.979921060516889, 'beta_t_minus_1')])